Make sure to mount your Google Drive or upload the necessary files. Paths are defined throughout and should be changed based on the personal filepaths of the user running the notebook

In [1]:
# Collective Imports

import pandas as pd
import re
from bs4 import BeautifulSoup
import urllib.request as urllib
import time
import os
from os.path import exists
import json
from datetime import datetime

In [4]:
# Use this if you're skipping steps to load the current top100_df
top100_df = pd.read_csv("/final-project-once-upon-a-vis/Data/cleanedall_reviews.csv")

FileNotFoundError: [Errno 2] File /final-project-once-upon-a-vis/Data/all_reviews.csv does not exist: '/final-project-once-upon-a-vis/Data/all_reviews.csv'

## 0. Collect and Clean our initial Dataset of 100 Books

In [ ]:
# Read and show the top 100 books data
top100_df = pd.read_csv("/content/drive/MyDrive/DS4200 Books/Top 100 books - TOP 100 - ALL TIME.csv", header=1, index_col=1)
top100_df.head()

,Index,ISBN,Title,Author,Imprint,Publisher Group,Volume,Value,RRP,ASP,Binding,Publ Date,Product Class
Position,,,,,,,,,,,,,
1,1,9780552149518,"Da Vinci Code,The","Brown, Dan",Corgi Books,Transworld Grp,"4,522,025","£22,857,837.53",£7.99,£5.05,Paperback,1 Mar 2004,"F2.1 Crime, Thriller & Adventure"
2,3,9780747532743,Harry Potter and the Philosopher's Stone#,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,844,316","£19,853,187.43",£6.99,£5.22,Paperback,26 Jun 1997,Y2.1 Children's Fiction
3,7,9780747538486,Harry Potter and the Chamber of Secrets#,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,184,492","£16,224,021.98",£6.99,£5.07,Paperback,1 Apr 1999,Y2.1 Children's Fiction
4,6,9780552150736,Angels and Demons#,"Brown, Dan",Corgi Books,Transworld Grp,"3,096,850","£15,537,324.84",£7.99,£5.05,Paperback,1 Jul 2003,"F2.1 Crime, Thriller & Adventure"
5,2,9780747551003,Harry Potter and the Order of the Phoenix,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,043,226","£33,925,431.19",£16.99,£11.15,Hardback,21 Jun 2003,Y2.1 Children's Fiction


In [ ]:
def clean_title(title):
  """
  Clean the Title of each book. This is important for the Goodreads scraping 
  we have to do below.
  
  Input:
    Any title - the value in the "Title" Column
  Output:
    str title - the cleaned title string after replacement 
                and string opperations
  """
  # change to a string to transform as needed
  try:
    title = title.to_string()
  except AttributeError:
    pass

  # get rid of excess symbols and whitespace
  title = re.sub("#","", title)
  title = re.sub("\n ", " ", title)
  title = re.sub("\n", " ",title)
  title = title.strip()

  return title


In [ ]:
# Apply the title cleaning function on our dataset's titles!
top100_df["Title"] = top100_df["Title"].apply(clean_title)
top100_df.head()

,Index,ISBN,Title,Author,Imprint,Publisher Group,Volume,Value,RRP,ASP,Binding,Publ Date,Product Class
Position,,,,,,,,,,,,,
1,1,9780552149518,"Da Vinci Code,The","Brown, Dan",Corgi Books,Transworld Grp,"4,522,025","£22,857,837.53",£7.99,£5.05,Paperback,1 Mar 2004,"F2.1 Crime, Thriller & Adventure"
2,3,9780747532743,Harry Potter and the Philosopher's Stone,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,844,316","£19,853,187.43",£6.99,£5.22,Paperback,26 Jun 1997,Y2.1 Children's Fiction
3,7,9780747538486,Harry Potter and the Chamber of Secrets,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,184,492","£16,224,021.98",£6.99,£5.07,Paperback,1 Apr 1999,Y2.1 Children's Fiction
4,6,9780552150736,Angels and Demons,"Brown, Dan",Corgi Books,Transworld Grp,"3,096,850","£15,537,324.84",£7.99,£5.05,Paperback,1 Jul 2003,"F2.1 Crime, Thriller & Adventure"
5,2,9780747551003,Harry Potter and the Order of the Phoenix,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,043,226","£33,925,431.19",£16.99,£11.15,Hardback,21 Jun 2003,Y2.1 Children's Fiction


## 1. Scrape to get Book Ids

In [ ]:
def scrape_goodreads_id(search):
  """
  Get the Goodreads ID for the first goodreads search result for a
  given search.

  Input:
    string search - the search term to use
  Output:
    string book_id - the book id for the first result gotten
                     from the search term
  """
  # get rid of the excess spaces in the search title
  search = re.sub(":", " ",search)
  search = re.sub(" ", "_",search)
  # remove items at the End of titles to ensure results
  search = re.sub(",The", "",search)
  search = re.sub(",A", "",search)
  search = re.sub("S\.", "",search)

  # get the html from a search on Goodreads for a book titled the search term
  html = urllib.urlopen("https://www.goodreads.com/search?utf8=%E2%9C%93&q=" 
                      + search 
                      + "&search_type=books")
  # turn the html into a BeautifulSoup object
  soup = BeautifulSoup(html.read())

  try:
    # get the url of the first book result from our search
    book_url = soup.body.table.tr.td.a.get("href")

    # add in a pause to reduce the chance of being blocked for scraping
    time.sleep(3)

    # replace a '?' in the first book url to help us get the goodreads book ID
    book_url = book_url.replace("?", "/")
    # return the book ID gotten from splitting "/show/book/__ID__/..." in the url
    book_id = book_url.split("/")[3]
    return book_id
  except:
    print("There were no results for the search:",search)
    return None

In [ ]:
# Add a column for each row with the goodreads ID
top100_df["Goodreads_ID"] = top100_df["Title"].apply(scrape_goodreads_id)

Harry_Potter_and_the_Half-blood_Prince_Children's_Edition
Dr._Atkins'_New_Diet_Revolution_The_No-hunger,_Luxurious_Weight_Loss_P
Delia's_How_to_Cook_(Bk.1)
Delia's_How_to_Cook_(Bk.2)
"Times"_Su_Doku_The_Utterly_Addictive_Number-placing_Puzzle_(Bk._1


In [ ]:
# drop the unattainable books
top100_df.dropna(inplace=True)

In [ ]:
top100_df["Goodreads_ID"].head()

Position
1                             968.The_Da_Vinci_Code
2    72193.Harry_Potter_and_the_Philosopher_s_Stone
3     15881.Harry_Potter_and_the_Chamber_of_Secrets
4                                 960.Angels_Demons
5       2.Harry_Potter_and_the_Order_of_the_Phoenix
Name: Goodreads_ID, dtype: object

## 2. Use Maria Antoniak's Goodreads Scraper [here](https://github.com/maria-antoniak/goodreads-scraper) to collect Goodreads metadata for each Book

In [ ]:
# clone the goodreads scraper (this makes a temporary download in your current
#   session of google drive)
!git clone https://github.com/maria-antoniak/goodreads-scraper.git

In [ ]:
# install all of the requirements
!pip install -r "/content/goodreads-scraper/requirements.txt"

In [ ]:
# make a temporary directory for the book metadata
!mkdir top100_book_metadata

In [ ]:
# set the goodreads ID as the index for the books and remove duplicates
#   (Surprise! We had different versions of the same Harry Potter books)
top100_df = top100_df.set_index("Goodreads_ID")
top100_df = top100_df[~top100_df.index.duplicated(keep='first')]

In [ ]:
# export a temporary csv of the goodreads IDs that are in the "Goodreads_ID" column
top100_df.to_csv("top100_book_ids.txt", columns=["Goodreads_ID"], header=False, index=False)

In [ ]:
# run the get_books.py file included in the github repo, which gets the 
#   goodreads metadata for the IDs we have in the DF/exported to a csv
#   temporarily in the above block
!python goodreads-scraper/get_books.py --book_ids_path top100_book_ids.txt --output_directory_path top100_book_metadata --format csv

2022-02-24 04:26:18.386875 get_books.py:

🎉 Success! All book metadata scraped. 🎉

Metadata files have been output to /top100_book_metadata
Goodreads scraping run time = ⏰ 0:00:00.073943 ⏰


## 3. Use Maria Antoniak's Goodreads Scraper [here](https://github.com/maria-antoniak/goodreads-scraper) to collect review data in a directory

This step **CAN NOT** be completed in Google Colab. Please use a different environment

In [ ]:
# make a temporary directory for the reviews to be stored
!mkdir top100_book_reviews

In [ ]:
# run the Goodreads scraper for the reviews using the Goodreads IDs we 
#   defined above for the metadata scraping
!python goodreads-scraper/get_reviews.py --book_ids_path /content/goodreads-scraper/test_book_ids.txt \
--output_directory_path top100_book_reviews --sort_order default --browser chrome 

##4. Add Goodreads metadata to our Dataframe and Download the updated CSV

In [ ]:
# Add the Goodreads metadata columns (these are values we don't have yet!)
top100_df["book series"] = 0
top100_df["num pages"] = 0
top100_df["genres"] = 0
top100_df["genres"] = top100_df["genres"].astype(object)
top100_df["shelves"] = 0
top100_df["shelves"] = top100_df["shelves"].astype(object)
top100_df["num goodreads ratings"] = 0
top100_df["average goodreads ratings"] = 0
top100_df["goodreads rating distribution"] = 0
top100_df["goodreads rating distribution"] = top100_df["goodreads rating distribution"].astype(object)
top100_df["num goodreads reviews"] = 0

In [ ]:
top100_df.head()

,Index,ISBN,Title,Author,Imprint,Publisher Group,Volume,Value,RRP,ASP,Binding,Publ Date,Product Class,book series,num pages,genres,shelves,num goodreads ratings,average goodreads ratings,goodreads rating distribution,num goodreads reviews
Goodreads_ID,,,,,,,,,,,,,,,,,,,,,
968.The_Da_Vinci_Code,1,9780552149518,"Da Vinci Code,The","Brown, Dan",Corgi Books,Transworld Grp,"4,522,025","£22,857,837.53",£7.99,£5.05,Paperback,1 Mar 2004,"F2.1 Crime, Thriller & Adventure",0,0,0,0,0,0,0,0
72193.Harry_Potter_and_the_Philosopher_s_Stone,3,9780747532743,Harry Potter and the Philosopher's Stone,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,844,316","£19,853,187.43",£6.99,£5.22,Paperback,26 Jun 1997,Y2.1 Children's Fiction,0,0,0,0,0,0,0,0
15881.Harry_Potter_and_the_Chamber_of_Secrets,7,9780747538486,Harry Potter and the Chamber of Secrets,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,184,492","£16,224,021.98",£6.99,£5.07,Paperback,1 Apr 1999,Y2.1 Children's Fiction,0,0,0,0,0,0,0,0
960.Angels_Demons,6,9780552150736,Angels and Demons,"Brown, Dan",Corgi Books,Transworld Grp,"3,096,850","£15,537,324.84",£7.99,£5.05,Paperback,1 Jul 2003,"F2.1 Crime, Thriller & Adventure",0,0,0,0,0,0,0,0
2.Harry_Potter_and_the_Order_of_the_Phoenix,2,9780747551003,Harry Potter and the Order of the Phoenix,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,043,226","£33,925,431.19",£16.99,£11.15,Hardback,21 Jun 2003,Y2.1 Children's Fiction,0,0,0,0,0,0,0,0


In [ ]:
# Mark the temporary directory where the book metadata has been stored
# IMPORTANT: delete the all books csv and json
directory = "/content/top100_book_metadata"

# For each file/book metadata collected in that temp directory
for filename in os.listdir(directory):
  f = os.path.join(directory, filename)

  # check if the file exists
  if (exists(f)):
    # open the json file
    with open(f) as file:
      book = json.load(file)
    
  # add the new goodreads metadata to our DataFrame (this causes warnings RIP)
  id = book["book_id_title"]
  top100_df.loc[id,"book series"] = book["book_series"]
  top100_df.loc[id,"num pages"] = book["num_pages"]
  top100_df.loc[id,"genres"] = [{"genres":book["genres"]}] # Change to Tuple? Set? this works for now 
  top100_df.loc[id,"shelves"] = [book["shelves"]]
  top100_df.loc[id,"num goodreads ratings"] = book["num_ratings"]
  top100_df.loc[id,"average goodreads ratings"] = book["average_rating"]
  top100_df.loc[id,"goodreads rating distribution"] = [book["rating_distribution"]]
  top100_df.loc[id,"num goodreads reviews"] = book["num_reviews"]


In [ ]:
top100_df.head()

,Index,ISBN,Title,Author,Imprint,Publisher Group,Volume,Value,RRP,ASP,Binding,Publ Date,Product Class,book series,num pages,genres,shelves,num goodreads ratings,average goodreads ratings,goodreads rating distribution,num goodreads reviews
Goodreads_ID,,,,,,,,,,,,,,,,,,,,,
968.The_Da_Vinci_Code,1,9780552149518,"Da Vinci Code,The","Brown, Dan",Corgi Books,Transworld Grp,"4,522,025","£22,857,837.53",£7.99,£5.05,Paperback,1 Mar 2004,"F2.1 Crime, Thriller & Adventure",Robert Langdon #2,489,"{'genres': ['Fiction', 'Mystery', 'Thriller', ...","{'to-read': 416339, 'currently-reading': 19278...",2121534,3.88,"{'5 Stars': 731457, '4 Stars': 729405, '3 Star...",50199
72193.Harry_Potter_and_the_Philosopher_s_Stone,3,9780747532743,Harry Potter and the Philosopher's Stone,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,844,316","£19,853,187.43",£6.99,£5.22,Paperback,26 Jun 1997,Y2.1 Children's Fiction,Harry Potter #1,223,"{'genres': ['Fantasy', 'Fiction', 'Young Adult...","{'to-read': 1266232, 'currently-reading': 1870...",8334489,4.48,"{'5 Stars': 5420633, '4 Stars': 1906445, '3 St...",131608
15881.Harry_Potter_and_the_Chamber_of_Secrets,7,9780747538486,Harry Potter and the Chamber of Secrets,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,184,492","£16,224,021.98",£6.99,£5.07,Paperback,1 Apr 1999,Y2.1 Children's Fiction,Harry Potter #2,341,"{'genres': ['Fantasy', 'Fiction', 'Young Adult...","{'fantasy': 57513, 'favorites': 35022, 'fictio...",3214825,4.43,"{'5 Stars': 1906462, '4 Stars': 878809, '3 Sta...",63057
960.Angels_Demons,6,9780552150736,Angels and Demons,"Brown, Dan",Corgi Books,Transworld Grp,"3,096,850","£15,537,324.84",£7.99,£5.05,Paperback,1 Jul 2003,"F2.1 Crime, Thriller & Adventure",Robert Langdon #1,736,"{'genres': ['Fiction', 'Mystery', 'Thriller', ...","{'to-read': 656905, 'currently-reading': 16603...",2921948,3.92,"{'5 Stars': 1044923, '4 Stars': 989925, '3 Sta...",31070
2.Harry_Potter_and_the_Order_of_the_Phoenix,2,9780747551003,Harry Potter and the Order of the Phoenix,"Rowling, J. K.",Bloomsbury Publishing PLC,Bloomsbury Grp,"3,043,226","£33,925,431.19",£16.99,£11.15,Hardback,21 Jun 2003,Y2.1 Children's Fiction,Harry Potter #5,870,"{'genres': ['Fantasy', 'Young Adult', 'Fiction...","{'to-read': 400226, 'currently-reading': 62033...",2882627,4.50,"{'5 Stars': 1842775, '4 Stars': 726288, '3 Sta...",50446


In [ ]:
top100_df.to_csv("/content/drive/MyDrive/DS4200 Books/top100_goodreads.csv")

##5. Turn Review Data into CSVs

In [ ]:
# Mark the directory where the reviews have been stored
directory = "/content/drive/MyDrive/DS4200 Books/reviews"
# Mark the directory where the CSVs will be saved
directory2 = "/content/drive/MyDrive/DS4200 Books/CSVreviews"

# For each file/book reviews collected in that temp directory
for filename in os.listdir(directory):
  f = os.path.join(directory, filename)
  f2 = os.path.join(directory2, filename.replace(".json",".csv"))

  # check if the file exists
  if (exists(f)):
    # open the json file
    with open(f) as file:
      review = json.load(file)
  
  # load the json of the 0 to 300 reviews for the given book
  df = pd.DataFrame(review)

  # remove duplicates
  df[["book_title","text", "user_name", "user_url"]] = df[["book_title","text", "user_name", "user_url"]].drop_duplicates(keep='first')
  df[["review_id"]] = df[["review_id"]].drop_duplicates(keep='first')

  # remove blank reviews
  df.dropna(subset = ["text"], inplace=True)

  # if it's not the all_reviews file, remove the book title and id elements
  #  from the DF
  if filename.startswith("all_reviews") is False:
    df.drop("book_id_title", axis=1, inplace=True)

  # remove the review url, user informaiton, and sorting method from our DF
  df.drop("review_url", axis=1, inplace=True)
  df.drop("user_name", axis=1, inplace=True)
  df.drop("user_url", axis=1, inplace=True)
  df.drop("sort_order", axis=1, inplace=True)
  df.drop("book_id", axis=1, inplace=True)
  df.drop("book_title", axis=1, inplace=True)

  # set the index to the review id
  df = df.set_index("review_id")

  # export the csv into our CSV directory
  df.to_csv(f2)

In [ ]:
pd.read_csv("/content/drive/MyDrive/DS4200 Books/CSVreviews/1110.The_Broker.csv").head()

,review_id,date,rating,text,num_likes,shelves
0,732356791,"Oct 02, 2013",4.0,"The Broker, John GrishamThe Broker is a suspen...",184,"['fiction', 'united-states', '21th-century', '..."
1,500811121,"May 02, 2019",2.0,I don't have a whole lot to say about this boo...,100,"['library', 'political-thrillers', 'audio', 'l..."
2,2321441749,"Apr 27, 2021",3.0,A stand alone thriller published 2005.A so-so ...,30,"['2021-reads', 'stand-alone-thrillers']"
3,3320175258,"Aug 24, 2021",4.0,Not exactly your typical Grisham legal-eagle b...,31,"['owned', 'legal-thriller']"
4,2031230279,"Jun 21, 2017",4.0,I finally found a Grisham novel that I don't r...,27,"['fiction', 'john-grisham']"


In [ ]:
pd.read_csv("/content/drive/MyDrive/DS4200 Books/CSVreviews/all_reviews.csv").head()

,review_id,book_id_title,date,rating,text,num_likes,shelves
0,732356791,1110.The_Broker,"Oct 02, 2013",4.0,"The Broker, John GrishamThe Broker is a suspen...",184,"['fiction', 'united-states', '21th-century', '..."
1,500811121,1110.The_Broker,"May 02, 2019",2.0,I don't have a whole lot to say about this boo...,100,"['library', 'political-thrillers', 'audio', 'l..."
2,2321441749,1110.The_Broker,"Apr 27, 2021",3.0,A stand alone thriller published 2005.A so-so ...,30,"['2021-reads', 'stand-alone-thrillers']"
3,3320175258,1110.The_Broker,"Aug 24, 2021",4.0,Not exactly your typical Grisham legal-eagle b...,31,"['owned', 'legal-thriller']"
4,2031230279,1110.The_Broker,"Jun 21, 2017",4.0,I finally found a Grisham novel that I don't r...,27,"['fiction', 'john-grisham']"


##6. Adding [Vader](https://github.com/cjhutto/vaderSentiment) Sentiment Analysis on Reviews (following [this](https://medium.com/@b.terryjack/nlp-pre-trained-sentiment-analysis-1eb52a9d742c) article)

In [ ]:
# Install missing library
!pip install twython

In [ ]:
# Import the pretrained sentiment analysis model 'Vader' from the python
#   library nltk
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [ ]:
#Example
sid.polarity_scores("I really like you but you can be so dumb sometimes")

{'compound': -0.7154, 'neg': 0.372, 'neu': 0.508, 'pos': 0.12}

In [ ]:
# Run it on each file!
# Mark the directory where the reviews CSVs have been stored
directory = "/content/drive/MyDrive/DS4200 Books/CSVreviews"

# For each file/book reviews collected in that temp directory
for filename in os.listdir(directory):
  f = os.path.join(directory, filename)
  
  # load the json of the 0 to 300 reviews for the given book
  df = pd.read_csv(f)
  df = df.dropna() # account for nan values
  
  # make new sentiment columns for the:
  df["Vader_score"] = -100000 # overall calculated score
  df["Vader_neg"] = -100000 # negativity
  df["Vader_neu"] = -100000 # neutrality
  df["Vader_pos"] = -100000 # positivity

  for row in df.index:
    try:
      sentiment = sid.polarity_scores(df.loc[row, "text"])
      df.loc[row, "Vader_score"] = sentiment["compound"]
      df.loc[row, "Vader_neg"] = sentiment["neg"]
      df.loc[row, "Vader_neu"] = sentiment["neu"]
      df.loc[row, "Vader_pos"] = sentiment["pos"]
    except:
      # print the row and the text value if the book review can't be analyzed
      print(row, df.loc[row, "text"])

  # export the analyzed csv into our CSV directory
  df.to_csv(f)

78 nan
44 nan
92 nan
248 nan
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
16 nan
17 nan
18 nan
19 nan
20 nan
21 nan
22 nan
23 nan
24 nan
25 nan
26 nan
27 nan
28 nan
29 nan
30 nan
31 nan
32 nan
33 nan
34 nan
35 nan
36 nan
37 nan
38 nan
39 nan
0 nan
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
16 nan
17 nan
18 nan
0 nan
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
0 nan
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
16 nan
17 nan
18 nan
19 nan
20 nan
21 nan
22 nan
23 nan
6068 nan
6069 nan
6070 nan
6071 nan
6072 nan
6073 nan
6074 nan
6075 nan
6076 nan
6077 nan
6078 nan
6079 nan
6080 nan
6081 nan
6082 nan
6083 nan
6084 nan
6085 nan
6086 nan
10415 nan
10463 nan
10619 nan
10670 nan
10671 nan
10672 nan
10673 nan
10674 nan
10675 nan
10676 nan
10677 nan
10678 nan
10679 nan
10680 nan
10681 nan
106

In [ ]:
pd.read_csv("/content/drive/MyDrive/DS4200 Books/CSVreviews/1110.The_Broker.csv").head()

,Unnamed: 0,review_id,date,rating,text,num_likes,shelves,Vader_score,Vader_neg,Vader_neu,Vader_pos
0,0,732356791,"Oct 02, 2013",4.0,"The Broker, John GrishamThe Broker is a suspen...",184,"['fiction', 'united-states', '21th-century', '...",0.8155,0.036,0.906,0.058
1,1,500811121,"May 02, 2019",2.0,I don't have a whole lot to say about this boo...,100,"['library', 'political-thrillers', 'audio', 'l...",0.7307,0.048,0.865,0.087
2,2,2321441749,"Apr 27, 2021",3.0,A stand alone thriller published 2005.A so-so ...,30,"['2021-reads', 'stand-alone-thrillers']",0.8884,0.058,0.844,0.097
3,3,3320175258,"Aug 24, 2021",4.0,Not exactly your typical Grisham legal-eagle b...,31,"['owned', 'legal-thriller']",0.7003,0.000,0.892,0.108
4,4,2031230279,"Jun 21, 2017",4.0,I finally found a Grisham novel that I don't r...,27,"['fiction', 'john-grisham']",0.8264,0.098,0.766,0.136


In [ ]:
pd.read_csv("/content/drive/MyDrive/DS4200 Books/CSVreviews/all_reviews.csv").head()

,Unnamed: 0,review_id,book_id_title,date,rating,text,num_likes,shelves,Vader_score,Vader_neg,Vader_neu,Vader_pos
0,0,732356791,1110.The_Broker,"Oct 02, 2013",4.0,"The Broker, John GrishamThe Broker is a suspen...",184,"['fiction', 'united-states', '21th-century', '...",0.8155,0.036,0.906,0.058
1,1,500811121,1110.The_Broker,"May 02, 2019",2.0,I don't have a whole lot to say about this boo...,100,"['library', 'political-thrillers', 'audio', 'l...",0.7307,0.048,0.865,0.087
2,2,2321441749,1110.The_Broker,"Apr 27, 2021",3.0,A stand alone thriller published 2005.A so-so ...,30,"['2021-reads', 'stand-alone-thrillers']",0.8884,0.058,0.844,0.097
3,3,3320175258,1110.The_Broker,"Aug 24, 2021",4.0,Not exactly your typical Grisham legal-eagle b...,31,"['owned', 'legal-thriller']",0.7003,0.000,0.892,0.108
4,4,2031230279,1110.The_Broker,"Jun 21, 2017",4.0,I finally found a Grisham novel that I don't r...,27,"['fiction', 'john-grisham']",0.8264,0.098,0.766,0.136


##7. Adding [TextBlob](https://textblob.readthedocs.io/en/dev/) Subjectivity Analysis on Reviews  (following [this](https://medium.com/@b.terryjack/nlp-pre-trained-sentiment-analysis-1eb52a9d742c) article)

In [ ]:
from textblob import TextBlob
# Example
TextBlob("I really like you but you can be so dumb sometimes").sentiment

Sentiment(polarity=-0.0875, subjectivity=0.35)

In [ ]:
# Run it on each file!
# Mark the directory where the reviews CSVs have been stored
directory = "/content/drive/MyDrive/DS4200 Books/CSVreviews"

# For each file/book reviews collected in that temp directory
for filename in os.listdir(directory):
  f = os.path.join(directory, filename)
  
  # load the json of the 0 to 300 reviews for the given book
  df = pd.read_csv(f)
  df = df.dropna() # account for nan values

  # make new sentiment columns for:
  df["TextBlob_sub"] = None # subjectivity

  for row in df.index:
    try: #TODO drop nans!
      sub = TextBlob(df.loc[row, "text"]).sentiment[1]
      df.loc[row, "TextBlob_sub"] = sub
    except:
      # print the row and the text value if the book review can't be analyzed
      print(row, df.loc[row, "text"])

  # export the analyzed csv into our CSV directory
  df.to_csv(f)

78 nan
44 nan
92 nan
248 nan
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
16 nan
17 nan
18 nan
19 nan
20 nan
21 nan
22 nan
23 nan
24 nan
25 nan
26 nan
27 nan
28 nan
29 nan
30 nan
31 nan
32 nan
33 nan
34 nan
35 nan
36 nan
37 nan
38 nan
39 nan
0 nan
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
16 nan
17 nan
18 nan
0 nan
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
0 nan
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
16 nan
17 nan
18 nan
19 nan
20 nan
21 nan
22 nan
23 nan
6068 nan
6069 nan
6070 nan
6071 nan
6072 nan
6073 nan
6074 nan
6075 nan
6076 nan
6077 nan
6078 nan
6079 nan
6080 nan
6081 nan
6082 nan
6083 nan
6084 nan
6085 nan
6086 nan
10415 nan
10463 nan
10619 nan
10670 nan
10671 nan
10672 nan
10673 nan
10674 nan
10675 nan
10676 nan
10677 nan
10678 nan
10679 nan
10680 nan
10681 nan
106

In [ ]:
pd.read_csv("/content/drive/MyDrive/DS4200 Books/CSVreviews/1110.The_Broker.csv").head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,review_id,date,rating,text,num_likes,shelves,Vader_score,Vader_neg,Vader_neu,Vader_pos,TextBlob_sub,Flair_score
0,0,0,0,732356791,"Oct 02, 2013",4.0,"The Broker, John GrishamThe Broker is a suspen...",184,"['fiction', 'united-states', '21th-century', '...",0.8155,0.036,0.906,0.058,0.384665,NEGATIVE (0.7892)
1,1,1,1,500811121,"May 02, 2019",2.0,I don't have a whole lot to say about this boo...,100,"['library', 'political-thrillers', 'audio', 'l...",0.7307,0.048,0.865,0.087,0.360185,NEGATIVE (1.0)
2,2,2,2,2321441749,"Apr 27, 2021",3.0,A stand alone thriller published 2005.A so-so ...,30,"['2021-reads', 'stand-alone-thrillers']",0.8884,0.058,0.844,0.097,0.413241,NEGATIVE (0.9998)
3,3,3,3,3320175258,"Aug 24, 2021",4.0,Not exactly your typical Grisham legal-eagle b...,31,"['owned', 'legal-thriller']",0.7003,0.000,0.892,0.108,0.547917,POSITIVE (0.5784)
4,4,4,4,2031230279,"Jun 21, 2017",4.0,I finally found a Grisham novel that I don't r...,27,"['fiction', 'john-grisham']",0.8264,0.098,0.766,0.136,0.511404,POSITIVE (0.9987)


In [ ]:
pd.read_csv("/content/drive/MyDrive/DS4200 Books/CSVreviews/all_reviews.csv").head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,review_id,book_id_title,date,rating,text,num_likes,shelves,Vader_score,Vader_neg,Vader_neu,Vader_pos,TextBlob_sub,Flair_score
0,0,0,0,732356791,1110.The_Broker,"Oct 02, 2013",4.0,"The Broker, John GrishamThe Broker is a suspen...",184,"['fiction', 'united-states', '21th-century', '...",0.8155,0.036,0.906,0.058,0.384665,NEGATIVE (0.7892)
1,1,1,1,500811121,1110.The_Broker,"May 02, 2019",2.0,I don't have a whole lot to say about this boo...,100,"['library', 'political-thrillers', 'audio', 'l...",0.7307,0.048,0.865,0.087,0.360185,NEGATIVE (1.0)
2,2,2,2,2321441749,1110.The_Broker,"Apr 27, 2021",3.0,A stand alone thriller published 2005.A so-so ...,30,"['2021-reads', 'stand-alone-thrillers']",0.8884,0.058,0.844,0.097,0.413241,NEGATIVE (0.9998)
3,3,3,3,3320175258,1110.The_Broker,"Aug 24, 2021",4.0,Not exactly your typical Grisham legal-eagle b...,31,"['owned', 'legal-thriller']",0.7003,0.000,0.892,0.108,0.547917,POSITIVE (0.5784)
4,4,4,4,2031230279,1110.The_Broker,"Jun 21, 2017",4.0,I finally found a Grisham novel that I don't r...,27,"['fiction', 'john-grisham']",0.8264,0.098,0.766,0.136,0.511404,POSITIVE (0.9987)


##8. Another Opinion: Adding [Flair](https://github.com/flairNLP/flair) Sentiment Analysis on Reviews  (following [this](https://medium.com/@b.terryjack/nlp-pre-trained-sentiment-analysis-1eb52a9d742c) article)

In [ ]:
!pip3 install flair

In [ ]:
import flair
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')

2022-03-01 03:56:06,153 loading file /root/.flair/models/sentiment-en-mix-distillbert_4.pt


In [ ]:
# Example
s = flair.data.Sentence("I really like you but you can be so dumb sometimes")
flair_sentiment.predict(s)
total_sentiment = s.labels
total_sentiment

[POSITIVE (0.9602)]

In [ ]:
# Run it on each file!
# Mark the directory where the reviews CSVs have been stored
directory = "/content/drive/MyDrive/DS4200 Books/CSVreviews"

# For each file/book reviews collected in that temp directory
for filename in os.listdir(directory):
  f = os.path.join(directory, filename)
  
  # load the json of the 0 to 300 reviews for the given book
  df = pd.read_csv(f)

  # make new sentiment columns for:
  df["Flair_score"] = None # sentiment

  for row in df.index:
    try: #TODO
      s = flair.data.Sentence(df.loc[row, "text"])
      flair_sentiment.predict(s)
      total_sentiment = s.labels[0]
      df.loc[row, "Flair_score"] = total_sentiment
    except:
      # print the row and the text value if the book review can't be analyzed
      print(row, df.loc[row, "text"])

  # export the analyzed csv into our CSV directory
  df.to_csv(f)

78 nan
44 nan
92 nan
248 nan
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
16 nan
17 nan
18 nan
19 nan
20 nan
21 nan
22 nan
23 nan
24 nan
25 nan
26 nan
27 nan
28 nan
29 nan
30 nan
31 nan
32 nan
33 nan
34 nan
35 nan
36 nan
37 nan
38 nan
39 nan
0 nan
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
16 nan
17 nan
18 nan
0 nan
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
29   
0 nan
1 nan
2 nan
3 nan
4 nan
5 nan
6 nan
7 nan
8 nan
9 nan
10 nan
11 nan
12 nan
13 nan
14 nan
15 nan
16 nan
17 nan
18 nan
19 nan
20 nan
21 nan
22 nan
23 nan
1028   
6068 nan
6069 nan
6070 nan
6071 nan
6072 nan
6073 nan
6074 nan
6075 nan
6076 nan
6077 nan
6078 nan
6079 nan
6080 nan
6081 nan
6082 nan
6083 nan
6084 nan
6085 nan
6086 nan
10415 nan
10463 nan
10619 nan
10670 nan
10671 nan
10672 nan
10673 nan
10674 nan
10675 nan
10676 nan
10677 nan
10678 nan
10679 nan
10680 nan

In [ ]:
pd.read_csv("/content/drive/MyDrive/DS4200 Books/CSVreviews/1110.The_Broker.csv").head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,review_id,date,rating,text,num_likes,shelves,Vader_score,Vader_neg,Vader_neu,Vader_pos,TextBlob_sub,Flair_score
0,0,0,0,732356791,"Oct 02, 2013",4.0,"The Broker, John GrishamThe Broker is a suspen...",184,"['fiction', 'united-states', '21th-century', '...",0.8155,0.036,0.906,0.058,0.384665,NEGATIVE (0.7892)
1,1,1,1,500811121,"May 02, 2019",2.0,I don't have a whole lot to say about this boo...,100,"['library', 'political-thrillers', 'audio', 'l...",0.7307,0.048,0.865,0.087,0.360185,NEGATIVE (1.0)
2,2,2,2,2321441749,"Apr 27, 2021",3.0,A stand alone thriller published 2005.A so-so ...,30,"['2021-reads', 'stand-alone-thrillers']",0.8884,0.058,0.844,0.097,0.413241,NEGATIVE (0.9998)
3,3,3,3,3320175258,"Aug 24, 2021",4.0,Not exactly your typical Grisham legal-eagle b...,31,"['owned', 'legal-thriller']",0.7003,0.000,0.892,0.108,0.547917,POSITIVE (0.5784)
4,4,4,4,2031230279,"Jun 21, 2017",4.0,I finally found a Grisham novel that I don't r...,27,"['fiction', 'john-grisham']",0.8264,0.098,0.766,0.136,0.511404,POSITIVE (0.9987)


In [ ]:
pd.read_csv("/content/drive/MyDrive/DS4200 Books/CSVreviews/all_reviews.csv").head()

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,review_id,book_id_title,date,rating,text,num_likes,shelves,Vader_score,Vader_neg,Vader_neu,Vader_pos,TextBlob_sub,Flair_score
0,0,0,0,732356791,1110.The_Broker,"Oct 02, 2013",4.0,"The Broker, John GrishamThe Broker is a suspen...",184,"['fiction', 'united-states', '21th-century', '...",0.8155,0.036,0.906,0.058,0.384665,NEGATIVE (0.7892)
1,1,1,1,500811121,1110.The_Broker,"May 02, 2019",2.0,I don't have a whole lot to say about this boo...,100,"['library', 'political-thrillers', 'audio', 'l...",0.7307,0.048,0.865,0.087,0.360185,NEGATIVE (1.0)
2,2,2,2,2321441749,1110.The_Broker,"Apr 27, 2021",3.0,A stand alone thriller published 2005.A so-so ...,30,"['2021-reads', 'stand-alone-thrillers']",0.8884,0.058,0.844,0.097,0.413241,NEGATIVE (0.9998)
3,3,3,3,3320175258,1110.The_Broker,"Aug 24, 2021",4.0,Not exactly your typical Grisham legal-eagle b...,31,"['owned', 'legal-thriller']",0.7003,0.000,0.892,0.108,0.547917,POSITIVE (0.5784)
4,4,4,4,2031230279,1110.The_Broker,"Jun 21, 2017",4.0,I finally found a Grisham novel that I don't r...,27,"['fiction', 'john-grisham']",0.8264,0.098,0.766,0.136,0.511404,POSITIVE (0.9987)


## 9. Clean up our review tables

In [ ]:
def change_date(date_str):
  date_str = str(date_str)
  try:
    date_obj = datetime.strptime(date_str, "%b\n%d,\n%Y")
    new_date_str = date_obj.strftime('%m/%d/%Y')
    return new_date_str
  except:
    print(date_str)
    return date_str

In [ ]:
# Example
change_date("Apr\n22,\n1989")

'04/22/1989'

In [ ]:
def Flair_name(flair):
  flair = str(flair)
  sent = flair.split(" ")[0]
  return sent

In [ ]:
# Example
Flair_name("NEGATIVE (0.7892)")

'NEGATIVE'

In [ ]:
def Flair_confidence(flair):
  try:
    flair = str(flair)
    sent = flair.split("(")[1]
    sent = sent.split(")")[0]
    return sent
  except:
    pass

In [ ]:
# Example
Flair_confidence("NEGATIVE (0.7892)")

'0.7892'

In [ ]:
# Run it on each file!
# Mark the directory where the reviews CSVs have been stored
directory = "/content/drive/MyDrive/DS4200 Books/CSVreviews"

# For each file/book reviews collected in that temp directory
for filename in os.listdir(directory):
  f = os.path.join(directory, filename)
  
  # load the json of the 0 to 300 reviews for the given book
  df = pd.read_csv(f)
  df = df.dropna() # account for nan values

  # clean date
  df["date"] = df["date"].apply(change_date)

  #clean Flair
  df["Flair_sent"] = df["Flair_score"].apply(Flair_name)
  df["Flair_conf"] = df["Flair_score"].apply(Flair_confidence)

  try: #TODO
    # You might need to do x number of the Unnamed drops depending on errors
    #   run into -> number of runs on the DF
    df.drop("Unnamed: 0", axis=1, inplace=True)
    df.drop("Unnamed: 0.1", axis=1, inplace=True)
    df.drop("Unnamed: 0.1.1", axis=1, inplace=True)
    df.drop("Unnamed: 0.1.1.1", axis=1, inplace=True)
    df.drop("shelves", axis=1, inplace=True)
    df.drop("Flair_score", axis=1, inplace=True)
  except:
    pass

  # export the analyzed csv into our CSV directory
  df.to_csv(f)

In [ ]:
pd.read_csv("/content/drive/MyDrive/DS4200 Books/CSVreviews/1110.The_Broker.csv").head()

,Unnamed: 0,review_id,date,rating,text,num_likes,Vader_score,Vader_neg,Vader_neu,Vader_pos,TextBlob_sub,Flair_sent,Flair_conf
0,0,2321441749,04/27/2021,3.0,A stand alone thriller published 2005.A so-so ...,30,0.8884,0.058,0.844,0.097,0.413241,NEGATIVE,0.9998
1,1,3320175258,08/24/2021,4.0,Not exactly your typical Grisham legal-eagle b...,31,0.7003,0.000,0.892,0.108,0.547917,POSITIVE,0.5784
2,2,2031230279,06/21/2017,4.0,I finally found a Grisham novel that I don't r...,27,0.8264,0.098,0.766,0.136,0.511404,POSITIVE,0.9987
3,3,524976279,02/03/2013,2.0,"It's OK. Not that bad, not that good either. T...",27,0.9272,0.127,0.681,0.192,0.554167,NEGATIVE,1.0000
4,4,1140586081,12/25/2014,2.0,It starts promisingly.. Then it gets repetitiv...,22,0.2263,0.074,0.781,0.145,0.583333,NEGATIVE,0.9997


In [ ]:
pd.read_csv("/content/drive/MyDrive/DS4200 Books/CSVreviews/all_reviews.csv").head()

,Unnamed: 0,review_id,book_id_title,date,rating,text,num_likes,Vader_score,Vader_neg,Vader_neu,Vader_pos,TextBlob_sub,Flair_sent,Flair_conf
0,0,2321441749,1110.The_Broker,04/27/2021,3.0,A stand alone thriller published 2005.A so-so ...,30,0.8884,0.058,0.844,0.097,0.413241,NEGATIVE,0.9998
1,1,3320175258,1110.The_Broker,08/24/2021,4.0,Not exactly your typical Grisham legal-eagle b...,31,0.7003,0.000,0.892,0.108,0.547917,POSITIVE,0.5784
2,2,2031230279,1110.The_Broker,06/21/2017,4.0,I finally found a Grisham novel that I don't r...,27,0.8264,0.098,0.766,0.136,0.511404,POSITIVE,0.9987
3,3,524976279,1110.The_Broker,02/03/2013,2.0,"It's OK. Not that bad, not that good either. T...",27,0.9272,0.127,0.681,0.192,0.554167,NEGATIVE,1.0000
4,4,1140586081,1110.The_Broker,12/25/2014,2.0,It starts promisingly.. Then it gets repetitiv...,22,0.2263,0.074,0.781,0.145,0.583333,NEGATIVE,0.9997
